# Expanding beam in a drift - zero current

The following notebook documents the free expansion of a beam in a drift, simulated with Synergia. A KV distribution is generated and traverses a 4m drift. The RMS beam properties are calculated and compare with theory.

### Import system libraries

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys, os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import tables
from mpi4py import MPI

### Import physics libraries

In [ ]:
try:
    import rssynergia
except ImportError:
    !pip -q install git+git://github.com/radiasoft/rssynergia

from rssynergia.base_diagnostics import read_bunch
from rssynergia.base_diagnostics import workflow
from rssynergia.base_diagnostics import lfplot
from rssynergia.base_diagnostics import latticework
from rssynergia.base_diagnostics import basic_calcs
from rssynergia.base_diagnostics import pltbunch
from rssynergia.base_diagnostics import elliptic_sp
from rssynergia.base_diagnostics import singleparticle
from rssynergia.base_diagnostics import options
from rssynergia.base_diagnostics import diagplot

from rssynergia.elliptic import elliptic_beam6d
from rssynergia.standard import standard_beam6d

import synergia
import synergia_workflow

### Create workdir and specify Synergia simulation options (default values)

In [ ]:
# Create and populate a Synergia options object
# File I/O
opts = synergia_workflow.Options("zc_drift")
opts.add("output_dir","zc_drift", "Directory for output files", str)
opts.relpath = opts.output_dir
workflow.make_path(opts.output_dir)
opts.add("verbosity", 1, "Verbosity of propagation", int)
opts.add("bunch_file","myBunch.txt","txt file for bunch particles", str)

# Define reference particle to be a proton at 2.5 MeV
total_energy = synergia.foundation.pconstants.proton_mass + 2.5e-3  # [GeV]
four_momentum = synergia.foundation.Four_momentum(synergia.foundation.pconstants.proton_mass, total_energy)
reference_particle = synergia.foundation.Reference_particle(synergia.foundation.pconstants.proton_charge,four_momentum)
opts.gamma = reference_particle.get_gamma()
opts.beta = reference_particle.get_beta()

# beam (physical)
opts.add("emit",9.74e-6, "H0 value corresponding to real sigma horizontal emittance of 0.3 mm-mrad", float)
opts.add("stdz", 0.05, "sigma read z [m]", float) #5 cm bunch length for IOTA
opts.add("dpop", 0.0, "Delta-p/p spread", float)
opts.add("real_particles", 1.0e11, "Number of real particles", float)
opts.emit_n = 0.3*1.e-6    # 0.3 mm-mrad normalized emittance
opts.emits = [basic_calcs.calc_geometric_emittance(opts.emit_n,opts.beta,opts.gamma)]
dpop = 0.0

# beam (numerical)
opts.add("macro_particles", 50000, "Number of macro particles", int)
opts.add("tracked_particles", 10000, "Number of tracked particles", int)
opts.add("seed", 349250524, "Pseudorandom number generator seed", int)
n_ppc = 100   # particles per cell (approximate)

# Space Charge
opts.add("gridx", 32, "grid points in x for solver", int)
opts.add("gridy", 32, "grid points in y for solver", int)
opts.add("gridz", 32, "grid points in z for solver", int)
opts.add("spacecharge", True, "whether space charge is on", bool)
opts.add("solver", "2dopen-hockney", "solver to use, '2dopen-hockney','3dopen-hockney', '2dbassetti-erskine'", str)

# Lattice
opts.add("steps_per_element",5,"Number of steps per element", int)
opts.add("turns",100,"Number of turns", int)
opts.add("radius", 0.5, "aperture radius [m]", float)
opts.add("stepper", "splitoperator", "Simulation stepper, either 'independent','elements','splitoperator','soelements'", str)

# MPI
opts.add("comm_divide", 8, "size of communicator")
opts.add("concurrent_io", 8, "number of concurrent io threads for checkpointing", int)
myrank = comm.get_rank()
mpisize = comm.get_size()

### Construct the lattice

In [ ]:
# specify the drift
my_drift_length = 0.02  # [m]
opts.steps_per_element = 2
drift_element = synergia.lattice.Lattice_element("drift", "o")
drift_element.set_double_attribute("l", my_drift_length)

# instantiate the lattice
lattice = synergia.lattice.Lattice("test", synergia.lattice.Mad8_adaptor_map())
lattice.append(drift_element)
lattice.set_reference_particle(reference_particle)
opts.lattice = lattice

# use a dummy operator
coll_operator = synergia.simulation.Dummy_collective_operator("dummy")

# instantiate the lattice stepper and simulator
map_order = 1
stepper = synergia.simulation.Split_operator_stepper_elements(lattice, map_order,coll_operator, opts.steps_per_element)
opts.lattice_simulator = stepper.get_lattice_simulator()

### Construct the bunch

In [ ]:
current = 14.e-3        # [A]
rp_perlength = current/(opts.beta*scipy.constants.c*scipy.constants.e)
bunch_length = 2*1.e-3  # effective bunch length [m]
opts.real_particles = rp_perlength*bunch_length
opts.betae = 1.0
opts.alphae = 0.0

particles = standard_beam6d.toyKVbeam6D(opts)
bunch = particles[0]
bunch[:,4] = bunch_length*(np.random.random(len(bunch)) -0.5) #center at 0
bunch[:,5] = opts.dpop*np.random.randn(1,len(bunch)) #set dp/p

# fix the weirdness with particle ID 4
bunch[4] = bunch[100]
bunch[4,6] = 4.0

particles_file = 'myKVBunch.txt'
np.savetxt('myKVBunch.txt',bunch)         # write the bunch to a text file

bucket_length = 1. #
comm = synergia.utils.Commxx(True) # define a communicator
myBunch = read_bunch.read_bunch(particles_file, reference_particle, opts.real_particles, bucket_length, comm)

# plot the initial distribution
pltbunch.plot_bunch(myBunch)

# save the initial distribution and calculate bunch properties
origBunch = myBunch
basic_calcs.calc_properties(origBunch,reference_particle)
gemitx = basic_calcs.get_emittance('x',origBunch)

### Run the simulation

In [ ]:
# instantiate a bunch simulator
bunch_simulator = synergia.simulation.Bunch_simulator(myBunch)

#basic diagnostics - PER STEP
basicdiag = synergia.bunch.Diagnostics_basic("basic.h5", opts.output_dir)
bunch_simulator.add_per_step(basicdiag)

#include full diagnostics
fulldiag = synergia.bunch.Diagnostics_full2("full.h5", opts.output_dir)
bunch_simulator.add_per_turn(fulldiag)

#particle diagnostics - PER TURN
opts.turnsPerDiag = 1
particlediag = synergia.bunch.Diagnostics_particles("particles.h5",0,0,opts.output_dir)
bunch_simulator.add_per_turn(particlediag, opts.turnsPerDiag)

# here the number of turns means the number of drifts
opts.turns = 200
opts.checkpointperiod = 50
opts.maxturns = opts.turns+1

# instantiate the propagator
propagator = synergia.simulation.Propagator(stepper)
propagator.set_checkpoint_period(opts.checkpointperiod)
propagator.propagate(bunch_simulator,opts.turns, opts.maxturns,opts.verbosity)

# clean up the output directory
workflow.cleanup(opts.output_dir)

## Diagnostics

#### Quick check of z trace space

In [ ]:
##Initial emittance
opts.relpath = opts.output_dir
files = elliptic_sp.get_file_list(opts)
twiss = elliptic_sp.get_toy_twiss(opts)
lost = elliptic_sp.get_lost_particle_list(opts)

if len(lost) > 0:
    #we have lost particles
    opts.lost = lost #store these in opts.lost
    lost = True #make lost a simple flag

#after turn1
outfile = files[1]
if lost:
    header, particles, lost_particles = elliptic_sp.get_particles(outfile, lost,opts.lost)
else:
    header, particles = elliptic_sp.get_particles(outfile, lost)

comm = synergia.utils.Commxx(True)
bucket_length = 10.
turn1Bunch = synergia.bunch.Bunch(
        reference_particle,
        particles.shape[0], opts.real_particles, comm,
        bucket_length)

local_num = turn1Bunch.get_local_num()
local_particles = turn1Bunch.get_local_particles()
local_particles[:,:]= particles[:,:]

#last bunch
outfile = files[-1]
if lost:
    header, particles, lost_particles = elliptic_sp.get_particles(outfile, lost,opts.lost)
else:
    header, particles = elliptic_sp.get_particles(outfile, lost)

comm = synergia.utils.Commxx(True)
bucket_length = 10.
endBunch = synergia.bunch.Bunch(
        reference_particle,
        particles.shape[0], opts.real_particles, comm,
        bucket_length)

local_num = endBunch.get_local_num()
local_particles = endBunch.get_local_particles()
local_particles[:,:]= particles[:,:]

In [ ]:
pltbunch.plot_bunch(endBunch)
pltbunch.plot_long(endBunch)

In [ ]:
opts.inputfile = opts.output_dir + '/basic.h5'
opts.plots = ['x_std', 'y_std', 'x_mean','y_mean']
plotVals = diagplot.getPlotVals(opts.inputfile, opts.plots)

#define specific value arrays
xmaster = plotVals['s']
xstd = plotVals['x_std']
ystd = plotVals['y_std']

#### First plot the envelope via the basic.h5 statistics

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.plot(xmaster,xstd*1.e3,'b-', alpha=0.7, label='xstd') #plot x
ax.plot(xmaster,ystd*1.e3,'g-', alpha=0.7, label='ystd') #plot y
#ax.plot(sval_0,xstd_100*1.e3,'g-',alpha=0.7, label='Turn 100') #plot the 1st turn
axtitle = "RMS envelope evolution over 4 m - Zero Current"
ax.set_title(axtitle, y = 1.02, fontsize = 18)  
ax.set_xlabel("s [m]",fontsize=14)
ax.set_ylabel("rms beam size $\sigma_x$ [mm]",fontsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.set_xlim([0,4.0])
ax.legend(loc=2)
sv_title = 'ZC_test_envelope.pdf'
fig.tight_layout()
#fig.savefig(sv_title,bbox_inches='tight')

#### Now plot using particle data

In [ ]:
opts.relpath = opts.output_dir
files = elliptic_sp.get_file_list(opts)
twiss = elliptic_sp.get_toy_twiss(opts)
lost = elliptic_sp.get_lost_particle_list(opts)

if len(lost) > 0:
    #we have lost particles
    opts.lost = lost #store these in opts.lost
    lost = True #make lost a simple flag


xrms_vals = []
    
for outfile in files:

    if lost:
        header, particles, lost_particles = elliptic_sp.get_particles(outfile, lost,opts.lost)
    else:
        header, particles = elliptic_sp.get_particles(outfile, lost)
    
    xrms = np.std(particles[:,0])
    
    xrms_vals.append(xrms)

zvals = (4/200)*np.asarray(list(range(201))) #construct s value parameters
xrms_vals = np.asarray(xrms_vals)

In [ ]:
#Compare the two
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.plot(xmaster,xstd*1.e3,'g-',alpha=0.5, label = 'simulation') #plot x
#ax.plot([p[0] for p in points], [p[1]*1.e3 for p in points], label = 'theory-14 mA')
ax.plot(zvals,xrms_vals*1.e3, 'b',alpha=0.5, label = 'np_std from particles.h5')
#ax.plot([p[0] for p in points2], [p[1]*1.e3 for p in points2],'k', label = 'theory - zero mA')
axtitle = "RMS envelope over 4m - zero current"
ax.set_title(axtitle, y = 1.02, fontsize = 18)  
ax.set_xlabel("s [m]",fontsize=14)
ax.set_ylabel("rms beam size $\sigma_x$ [mm]",fontsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.set_xlim([0,4.0])
ax.legend(loc = 2)
sv_title = 'ZC_test_envelope_compare.pdf'
fig.tight_layout()
#fig.savefig(sv_title,bbox_inches='tight')

They are in agreement! This is good.

#### Quick calculation of zero current expansion

In [ ]:
def calc_perveance(I,ref,cn=0):
    '''Calculate the perveance for a proton beam of a given current and particle energy.
    
    Arguments
        - I - current in A
        - ref - the reference particle for extracting beta and gamma
        
        - (optional) charge neutralization factor - default 0
    '''
    
    I0 = 3.13e7 #characteristic current
    
    beta = ref.get_beta()
    gamma = ref.get_gamma()
    
    return (I/I0)*(2/beta**3)*(1/gamma**3)

def calc_characteristic_current():
    '''Return characteristics current for proton beam'''
    return 4*np.pi*scipy.constants.epsilon_0*scipy.constants.m_p*(scipy.constants.c**3)/scipy.constants.e

In [ ]:
#Introduce numerical integrators
import math
from __future__ import division

#we are incrementing (stepping through z) I suppose
N = 400 #number of steps (e.g. 1 cm steps)
z0 = 0.0 #start
zf = 4.0 #end
ss = (zf-z0)/N #step size

zpoints = np.linspace(0.0, 4.0, num=N) #define z values
rpoints = [] #empty array for r values


#initial conditions
#current= 0.0005 
#current = 0.014/(400.)
current = 0
Kp = calc_perveance(current, reference_particle)
rprime0 = (xstd[1]-xstd[0])/(xmaster[1]-xmaster[0]) #need to seed with a nonzero initial rprime
r0 = xstd[0] #initial envelope value
#emit = hA[0,1] #emittance in m-rad -> geometric not normalized
emit = gemitx  #rms geometric emittance

#function here, which is a function of r and z
def rprime(K,emit,r0,rp0,rm):
    '''Returns the slope of the beam envelope (dr/dz) 
    for a given value of emittance,rm, K, and initial conditions'''
    
    first = rp0**2 #first term
    second = (emit**2)*((1./r0**2)-(1./rm**2)) #second term
    third = 2*K* np.log(rm/r0)
    
    return np.sqrt(first + second + third)


#x is r
#z is t (what we step up)
#f is our function describing the relationship between r and z
f = lambda r: rprime(Kp,emit,r0,rprime0,r)

#2nd Order RK - Ralston Method
def Ralston(r,z,h,f):
    k1 = h*f(r)
    return 0.25*k1 + 0.75*h*f(r+(2/3)*k1)

#4th Order Runge-Kutta
def RungeKutta4(r,z,h,f):
    k1 = f(r)
    k2 = f(r + (h/2)*k1)
    k3 = f(r + (h/2)*k2)
    k4 = f(r + h*k3)
    return h/6*(k1 + 2*k2 +2*k3 + k4)
 

In [ ]:
r,z,dz = r0,0,ss
points = []
while z < zf:
    points.append((z,r))
    z, r = z+dz, r + Ralston(r,z,dz,f) #incremement
    
r,z,dz = r0,0,ss
pointsRK4 = []
while z < zf:
    pointsRK4.append((z,r))
    z, r = z+dz, r + RungeKutta4(r,z,dz,f) #incremement


#Compare the integrators
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.plot([p[0] for p in pointsRK4], [p[1]*1.e3 for p in pointsRK4],'r--',alpha=1, label = 'RK4') #plot x
#ax.plot([p[0] for p in points], [p[1]*1.e3 for p in points], label = 'theory-14 mA')
#ax.plot(zvals,xrms_vals*1.e3, 'b',alpha=0.5, label = 'np_std from particles.h5')
ax.plot([p[0] for p in points], [p[1]*1.e3 for p in points],'b-',alpha=0.5, label = 'Ralston')
axtitle = "RMS envelope over 4m with zero current - integrator comparison"
ax.set_title(axtitle, y = 1.02, fontsize = 18)  
ax.set_xlabel("s [m]",fontsize=14)
ax.set_ylabel("rms beam size $\sigma_x$ [mm]",fontsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.set_xlim([0,4.0])
ax.legend(loc = 2)
sv_title = 'ZC_envelope_integrator_compare.pdf'
fig.tight_layout()
#fig.savefig(sv_title,bbox_inches='tight')

## Compare Theoretical Expansion to Simulated Expansion

In [ ]:
#Compare the two
fig = plt.figure(figsize=(8,6))
ax = plt.gca()
ax.plot(xmaster,xstd*1.e3,'r--',alpha=1, label = 'simulation') #plot x
#ax.plot([p[0] for p in points], [p[1]*1.e3 for p in points], label = 'theory-14 mA')
#ax.plot(zvals,xrms_vals*1.e3, 'b',alpha=0.5, label = 'np_std from particles.h5')
ax.plot([p[0] for p in points], [p[1]*1.e3 for p in points],'b-',alpha=0.5, label = 'theory')
axtitle = "RMS envelope over 4m - zero current"
ax.set_title(axtitle, y = 1.02, fontsize = 18)  
ax.set_xlabel("s [m]",fontsize=14)
ax.set_ylabel("rms beam size $\sigma_x$ [mm]",fontsize=14)
ax.tick_params(axis='x', labelsize=14)
ax.tick_params(axis='y', labelsize=14)
ax.set_xlim([0,4.0])
ax.legend(loc = 2)
sv_title = 'ZC_test_envelope_compare.pdf'
fig.tight_layout()
#fig.savefig(sv_title,bbox_inches='tight')

We see excellent agreement between the predicted and simulated expansion in the zero current limit.